In [2]:
import json

In [3]:
with open("HP_KG_7_books.json", "r", encoding="utf-8") as f:
    data = json.load(f)


# Step 2: Convert to a dictionary with numbered integer keys
def transform_to_numbered_dict(data):
    return {i: item for i, item in enumerate(data, start=1)}

numbered_data = transform_to_numbered_dict(data)

# Step 3: Write the transformed data to a new JSON file
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(numbered_data, f, indent=4)

In [10]:
import json

# Load your JSON data (replace this with your actual JSON loading)
with open('output.json', 'r') as f:
    data = json.load(f)

# Merge all sublists in "Nodes sequence" into one flat list per item
for key, value in data.items():
    merged_nodes = []
    seen_ids = set()
    for sequence in value.get("Nodes sequence", []):
        merged_nodes.extend(sequence)
    data[key]["Nodes sequence"] = merged_nodes

# Save or print the modified JSON
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)


In [12]:
import json

# Load your JSON
with open('output.json', 'r') as f:
    data = json.load(f)

# Fix "Nodes sequence" across all entries
for key, value in data.items():
    nodes_sequence = value.get("Nodes sequence", [])
    
    # Case 1: Flat list (wrong format)
    if all(isinstance(item, str) for item in nodes_sequence):
        corrected = []
        for i in range(0, len(nodes_sequence), 2):
            if i + 1 < len(nodes_sequence):
                corrected.append({
                    "id": nodes_sequence[i],
                    "name": nodes_sequence[i + 1]
                })
        data[key]["Nodes sequence"] = corrected

    # Case 2: List of sublists (check each sublist)
    elif all(isinstance(item, list) for item in nodes_sequence):
        corrected = []
        for sublist in nodes_sequence:
            if all(isinstance(item, str) for item in sublist):
                # Fix each flat sublist
                for i in range(0, len(sublist), 2):
                    if i + 1 < len(sublist):
                        corrected.append({
                            "id": sublist[i],
                            "name": sublist[i + 1]
                        })
            elif all(isinstance(item, dict) for item in sublist):
                corrected.extend(sublist)
        data[key]["Nodes sequence"] = corrected

    # Case 3: Already a list of dicts — do nothing
    elif all(isinstance(item, dict) for item in nodes_sequence):
        continue

# Save the corrected JSON
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)


In [13]:
import json

# Load JSON
with open('output.json', 'r') as f:
    data = json.load(f)

# Deduplicate each "Nodes sequence" by node["id"]
for key, value in data.items():
    seen_ids = set()
    unique_nodes = []
    
    for node in value.get("Nodes sequence", []):
        if isinstance(node, dict):
            node_id = node.get("id")
            if node_id and node_id not in seen_ids:
                unique_nodes.append(node)
                seen_ids.add(node_id)
    
    data[key]["Nodes sequence"] = unique_nodes

# Save the deduplicated result
with open('output.json', 'w') as f:
    json.dump(data, f, indent=4)


In [15]:
import json

# === Step 1: Load JSON files ===
with open("Special.json", "r", encoding="utf-8") as f:
    creatures_data = json.load(f)

with open("output.json", "r", encoding="utf-8") as f:
    scenes_data = json.load(f)

# === Step 2: Create a lookup for creatures by ID ===
creature_dict = {creature['id']: creature for creature in creatures_data}

# === Step 3: Initialize "list of appear" ===
for creature in creature_dict.values():
    creature['list of appear'] = []

# === Step 4: Populate appearances from scenes ===
for scene_num, scene in scenes_data.items():
    for node in scene.get("Nodes sequence", []):
        node_id = node["id"]
        # Only count direct creature IDs (no Pred_/Spec_ variants)
        if node_id in creature_dict:
            creature_dict[node_id]["list of appear"].append(int(scene_num))

# === Step 5: Save updated creatures to a new file ===
with open("Special_new.json", "w", encoding="utf-8") as f:
    json.dump(list(creature_dict.values()), f, indent=4, ensure_ascii=False)
